使用sklearn训练集训练逻辑回归模型

调整学习率，样本数据拆分比率，观察训练结果  
把模型训练参数保存到文件，在另一个代码中加载参数实现预测功能

In [4]:
import numpy as np
from sklearn.datasets import load_iris  # 导入鸢尾花数据集
from sklearn.model_selection import train_test_split

In [5]:
X,y = load_iris(return_X_y=True) # 加载鸢尾花数据集
print(X[0],y)
# print(X[0],y[0])
X.shape,y.shape  # 100个样本,4个特征值(花萼长度，花萼宽度，花瓣长度，花瓣宽度)

[5.1 3.5 1.4 0.2] [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


((150, 4), (150,))

In [6]:
X = X[:100]
y = y[:100]
X.shape,y.shape 

((100, 4), (100,))

In [7]:
# 数据拆分
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [8]:
# 参数初始化
theta = np.random.randn(1,4)
print('theta:',theta)
bias = 0
lr = 1e-1 
epochs = 10001
epsilon = 1e-8

theta: [[-0.9033504   1.0276371   0.08496017  0.39148379]]


### 模型构建

In [9]:
# 向前传播
def forward(X, theta, bias):
    z = np.dot(theta, X.T) + bias  # 计算线性部分
    y_hat = 1 / (1 + np.exp(-z))  # 计算激活函数sigmoid
    return y_hat


In [10]:
# 损失函数
def loss(y,y_hat):
    epsilon = 1e-15  # 添加一个极小值
    y_hat = np.clip(y_hat, epsilon, 1 - epsilon)  # 将 y_hat 限制在 [epsilon, 1 - epsilon] 范围内
    return -np.mean(y*np.log(y_hat)+(1-y)*np.log(1-y_hat))


In [11]:
# 反向传播，优化器，梯度下降法
def backward(X, y, y_hat, theta, bias, lr):
    m = X.shape[0]
    dz = y_hat - y
    dtheta = np.dot(dz, X) / m
    dbias = np.sum(dz) / m
    theta -= lr * dtheta
    bias -= lr * dbias
    return theta, bias

In [12]:
# 训练
for i in range(epochs):
    y_hat = forward(X_train, theta, bias)
    loss_val = loss(y_hat, y_train)
    if(abs(loss_val) < epsilon):
        break
    theta, bias = backward(X_train, y_train, y_hat ,theta, bias ,lr)
    if i % 1000 == 0:
        acc = np.mean(y_hat == y_train)
        print('Epoch: %d, loss: %.20f, accuracy: %.20f' % (i, loss_val, acc))

Epoch: 0, loss: 20.28051233970961320097, accuracy: 0.00000000000000000000
Epoch: 1000, loss: 0.28325223040300323563, accuracy: 0.00000000000000000000
Epoch: 2000, loss: 0.15041455778616230621, accuracy: 0.00000000000000000000
Epoch: 3000, loss: 0.10402409773819566718, accuracy: 0.00000000000000000000
Epoch: 4000, loss: 0.08010496221703358732, accuracy: 0.00000000000000000000
Epoch: 5000, loss: 0.06541615257549789064, accuracy: 0.00000000000000000000
Epoch: 6000, loss: 0.05543842315152772149, accuracy: 0.00000000000000000000
Epoch: 7000, loss: 0.04819863919898204063, accuracy: 0.00000000000000000000
Epoch: 8000, loss: 0.04269487244260795206, accuracy: 0.00000000000000000000
Epoch: 9000, loss: 0.03836305414081923187, accuracy: 0.00000000000000000000
Epoch: 10000, loss: 0.03486077004782774102, accuracy: 0.00000000000000000000


### 测试数据

In [13]:
index = np.random.randint(len(X_test))
x,y = X_test[index],y_test[index]

predict = np.round(forward(x,theta,bias))
print('预测值: ',predict , '\n真实值: ',y_test[index])

预测值:  [0.] 
真实值:  0


In [ ]:
# 保存 二进制文件
np.save('iris_X_test.npy',X_test)
np.save('iris_y_test.npy',y_test)
np.save('iris_theta.npy',theta)
np.save('iris_bias.npy',bias)
